# Scaling Training Data

In [1]:
import pandas as pd
import numpy as np
import random
random.seed(8)

In [19]:
pd.set_option('display.max_columns', None)

Loading case and subcohort training data and joining them.

In [2]:
cases_train = pd.read_csv("D:/compiled_data/cases_train.csv")

In [3]:
subcohort_train = pd.read_csv("D:/compiled_data/subcohort_train.csv")

In [24]:
case_subcohort = pd.concat([cases_train,subcohort_train]).set_index("csid")
case_subcohort

,qc_warning,precipitation,il8_x100000,tnfrsf9_x100000,tie2_x100000,mcp_3_x100000,cd40_l_x100000,il_1_alpha_x100000,cd244_x100000,egf_x100000,ang_1_x100000,il7_x100000,pgf_x100000,il6_x100000,adgrg1_x100000,mcp_1_x100000,crtam_x100000,cxcl11_x100000,mcp_4_x100000,trail_x100000,fgf2_x100000,cxcl9_x100000,cd8a_x100000,caix_x100000,ifn_beta_x100000,ada_x100000,cd4_x100000,nos3_x100000,il2_x100000,gal_9_x100000,vegfr_2_x100000,cd40_x100000,il18_x100000,gzmh_x100000,vegfc_x100000,lap_tgf_beta_1_x100000,cxcl1_x100000,tnfsf14_x100000,il33_x100000,tweak_x100000,pdgf_subunit_b_x100000,pdcd1_x100000,faslg_x100000,cd28_x100000,ccl19_x100000,mcp_2_x100000,ccl4_x100000,il_35_x100000,gal_1_x100000,pd_l1_x100000,cd27_x100000,cxcl5_x100000,il5_x100000,hgf_x100000,gzma_x100000,ho_1_x100000,cx3cl1_x100000,cxcl10_x100000,cd70_x100000,il10_x100000,tnfrsf12a_x100000,ccl23_x100000,cd5_x100000,ccl3_x100000,mmp7_x100000,arg1_x100000,ncr1_x100000,dcn_x100000,tnfrsf21_x100000,tnfrsf4_x100000,mic_a_b_x100000,ccl17_x100000,angpt2_x100000,ptn_x100000,cxcl12_x100000,ifn_gamma_x100000,lamp3_x100000,casp_8_x100000,icoslg_x100000,mmp12_x100000,cxcl13_x100000,pd_l2_x100000,vegfa_x100000,il4_x100000,il_21_x100000,il12rb1_x100000,il13_x100000,ccl20_x100000,tnf_x100000,klrd1_x100000,gzmb_x100000,cd83_x100000,il12_x100000,csf_1_x100000,region_is_urban,study_date,is_female,age_at_study_date_x100,household_size,has_health_cover,has_own_home,has_private_toilet,has_phone,has_motor_vehicle,had_recent_holiday,tea_drank_in_past,tea_years_ago,alc_today,alc_drank_in_past,alc_years_ago,smoking_today,smoking_today_amount,smoking_last_hour_amount,smoking_100_plus_cigs,has_diabetes,has_copd,diabetes_diag,diabetes_diag_age,diabetes_now,chd_diag,chd_diag_age,chd_now,stroke_or_tia_diag,stroke_or_tia_diag_age,hypertension_diag,stroke_or_tia_now,hypertension_diag_age,hypertension_now,taking_aspirin,taking_ace_i,taking_beta_blocker,taking_diuretics,taking_statins,taking_ca_antagonist,taking_chlor_metaformin,taking_insulin,rheum_heart_dis_diag,rheum_heart_dis_diag_age,rheum_heart_dis_now,tb_diag,tb_diag_age,tb_now,emph_bronc_diag,emph_bronc_diag_age,emph_bronc_now,asthma_diag,asthma_diag_age,asthma_now,cirrhosis_hep_diag,cirrhosis_hep_diag_age,cirrhosis_hep_now,peptic_ulcer_diag,peptic_ulcer_diag_age,peptic_ulcer_now,gall_diag,gall_diag_age,gall_now,kidney_dis_diag,kidney_dis_diag_age,kidney_dis_now,fracture_diag,fracture_diag_age,fracture_now,rheum_arthritis_diag,rheum_arthritis_diag_age,rheum_arthritis_now,psych_disorder_diag,psych_disorder_diag_age,psych_disorder_now,neurasthenia_diag,neurasthenia_diag_age,head_injury_diag,neurasthenia_now,head_injury_diag_age,head_injury_now,cancer_diag,cancer_diag_age,cancer_now,blood_transfusions,paid_blood_donations,children,siblings,mother_current_age,mother_age_at_death,father_current_age,father_age_at_death,mother_stroke,mother_heart_attack,mother_diabetes,mother_mental_disorder,father_stroke,mother_cancer,father_diabetes,father_heart_attack,father_mental_disorder,father_cancer,siblings_stroke,siblings_diabetes,siblings_heart_attack,siblings_cancer,siblings_mental_disorder,children_stroke,children_heart_attack,children_diabetes,children_cancer,children_mental_disorder,met,met_hours,met_work,met_work_hours,tried_to_reduce_weight,age_25_weight_jin,standing_height_mm,sitting_height_mm,waist_mm,hip_mm,waist_hip_ratio,weight_kg_x10,bmi_calc,impedance_ohms,fat_percent_x10,weight_etc_staff_code,used_blood_pressure_drugs,sbp_mean,dbp_mean,heart_rate_mean,sbp_first,dbp_first,heart_rate_first,sbp_second,dbp_second,heart_rate_second,blood_staff_code,hours_since_last_ate_x10,random_glucose_x10,fasting_glucose_x10,ep_CKB0020_combined_ep,ep_CKB0020_combined_datedeveloped,region_code_12,region_code_16,region_code_26,region_code_36,region_code_46,region_code_52,region_code_58,region_code_68,region_code_78,region_code_88,highest_education_0,highest_education_1,highest_education_2,highest_education_3,highest_education_4,highest_education_5,occupation_0,occupa

In [5]:
X = case_subcohort.drop(columns = ["ep_CKB0020_combined_ep","ep_CKB0020_combined_datedeveloped","study_date","subcohort"])
y = case_subcohort[["ep_CKB0020_combined_ep","ep_CKB0020_combined_datedeveloped","study_date","subcohort"]]

## Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

Need to find columns that require scaling:

In [7]:
column_data = case_subcohort.agg(["min","max"]).transpose()
column_data

,min,max
qc_warning,0,1
precipitation,0,1
il8_x100000,254254,1316276
tnfrsf9_x100000,315243,776254
tie2_x100000,587198,788241
...,...,...
weight_changed_2,0,1
hep_b_0.0,0,1
hep_b_1.0,0,1
hep_b_2.0,0,1


In [8]:
# Columns with now variation
no_var_cols = column_data.query("min == max").index
no_var_cols

Index(['rheum_heart_dis_diag_age', 'rheum_heart_dis_now', 'psych_disorder_now',
       'head_injury_now', 'children_stroke', 'children_heart_attack'],
      dtype='object')

In [9]:
# Dropping columns with no variation
X = X.drop(columns = no_var_cols)

column_data = column_data.drop(no_var_cols)

In [10]:
unscaled_cols = column_data.query("not( min == 0 and max == 1)").index
unscaled_cols

Index(['il8_x100000', 'tnfrsf9_x100000', 'tie2_x100000', 'mcp_3_x100000',
       'cd40_l_x100000', 'il_1_alpha_x100000', 'cd244_x100000', 'egf_x100000',
       'ang_1_x100000', 'il7_x100000',
       ...
       'dbp_first', 'heart_rate_first', 'sbp_second', 'dbp_second',
       'heart_rate_second', 'blood_staff_code', 'hours_since_last_ate_x10',
       'random_glucose_x10', 'fasting_glucose_x10',
       'ep_CKB0020_combined_datedeveloped'],
      dtype='object', length=156)

In [11]:
# the columns of X that need to be scaled
X_unscaled = X[X.columns[np.where(X.columns.isin(unscaled_cols))]]
X_cat = X[X.columns[np.where(~X.columns.isin(unscaled_cols))]]
X_unscaled

,il8_x100000,tnfrsf9_x100000,tie2_x100000,mcp_3_x100000,cd40_l_x100000,il_1_alpha_x100000,cd244_x100000,egf_x100000,ang_1_x100000,il7_x100000,...,sbp_first,dbp_first,heart_rate_first,sbp_second,dbp_second,heart_rate_second,blood_staff_code,hours_since_last_ate_x10,random_glucose_x10,fasting_glucose_x10
csid,,,,,,,,,,,,,,,,,,,,,
2022-00229-1799093,451202,512758,708194,232248,312714,42339,610982,287965,551375,290993,...,131,84,84,126,82,80,113,20,56.0,NaN
2022-00229-4744170,334234,465765,679716,170029,313600,42339,542825,355536,508725,256926,...,125,52,80,127,58,83,113,10,67.0,NaN
2022-00229-1082517,378818,475773,698859,121908,417786,42339,567657,534603,700434,267167,...,176,97,79,177,99,80,106,30,65.0,NaN
2022-00229-3397875,534710,521784,691196,190591,542269,42339,562981,694091,875492,361438,...,112,71,74,114,68,74,129,15,53.0,NaN
2022-00229-4372175,594503,563830,743231,233956,580853,42339,631639,685348,956935,664253,...,133,81,67,125,79,69,125,160,61.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-00229-5565114,477923,445954,688681,170687,331135,42339,526045,489770,722139,413903,...,120,73,80,116,72,79,128,20,65.0,NaN
2022-00229-4902412,348610,450344,693170,135809,249155,42339,561047,343273,496623,332210,...,113,77,92,109,78,88,111,20,44.0,NaN
2022-00229-4823882,653205,490802,732062,174762,554958,42339,569138,630823,874013,357443,...,124,69,57,116,75,61,108,20,64.0,NaN


In [12]:
# transforming the data using the scaler
scaler = StandardScaler()

X_scaled = pd.DataFrame(scaler.fit_transform(X_unscaled), index = X_unscaled.index, columns = X_unscaled.columns)
X_scaled

,il8_x100000,tnfrsf9_x100000,tie2_x100000,mcp_3_x100000,cd40_l_x100000,il_1_alpha_x100000,cd244_x100000,egf_x100000,ang_1_x100000,il7_x100000,...,sbp_first,dbp_first,heart_rate_first,sbp_second,dbp_second,heart_rate_second,blood_staff_code,hours_since_last_ate_x10,random_glucose_x10,fasting_glucose_x10
csid,,,,,,,,,,,,,,,,,,,,,
2022-00229-1799093,0.122746,0.811057,0.592906,1.728535,-0.410683,-0.056127,1.780844,-1.604290,-0.884395,-0.449261,...,-0.190927,0.434371,0.418815,-0.340807,0.356325,0.112450,-0.473569,-0.654764,-0.272600,NaN
2022-00229-4744170,-1.250938,-0.168420,-0.422900,0.301515,-0.402491,-0.056127,-0.277026,-1.096393,-1.183793,-0.787761,...,-0.464590,-2.338602,0.093618,-0.295146,-1.779377,0.357120,-0.473569,-0.847423,0.093850,NaN
2022-00229-1082517,-0.727339,0.040177,0.259928,-0.802161,0.560934,-0.056127,0.472728,0.249563,0.161982,-0.686003,...,1.861547,1.560891,0.012318,1.987918,1.869113,0.112450,-1.218312,-0.462106,0.027223,NaN
2022-00229-3397875,1.103472,0.999186,-0.013410,0.773113,1.712047,-0.056127,0.331545,1.448354,1.390868,0.250701,...,-1.057527,-0.692149,-0.394179,-0.888742,-0.889501,-0.376892,1.228701,-0.751094,-0.372541,NaN
2022-00229-4372175,1.805687,1.875552,1.842670,1.767708,2.068840,-0.056127,2.404542,1.382637,1.962589,3.259559,...,-0.099706,0.174405,-0.963275,-0.386468,0.089362,-0.784676,0.803133,2.042459,-0.106032,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-00229-5565114,0.436560,-0.581341,-0.103120,0.316606,-0.240342,-0.056127,-0.783666,-0.087424,0.314348,0.772009,...,-0.692643,-0.518838,0.093618,-0.797419,-0.533551,0.030893,1.122309,-0.654764,0.027223,NaN
2022-00229-4902412,-1.082104,-0.489840,0.057002,-0.483336,-0.998423,-0.056127,0.273152,-1.188568,-1.268748,-0.039717,...,-1.011916,-0.172217,1.069210,-1.117048,0.000375,0.764905,-0.686353,-0.654764,-0.672364,NaN
2022-00229-4823882,2.495089,0.353427,1.444274,0.410068,1.829385,-0.056127,0.517444,0.972800,1.380486,0.211005,...,-0.510201,-0.865460,-1.776269,-0.797419,-0.266588,-1.437132,-1.005528,-0.654764,-0.006091,NaN


In [13]:
# setting scaler NA values to 0, which should be the mean
imp_const = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = 0)
X_scaled_imp = pd.DataFrame(imp_const.fit_transform(X_scaled), index = X_scaled.index, columns = X_scaled.columns)
X_scaled_imp

,il8_x100000,tnfrsf9_x100000,tie2_x100000,mcp_3_x100000,cd40_l_x100000,il_1_alpha_x100000,cd244_x100000,egf_x100000,ang_1_x100000,il7_x100000,...,sbp_first,dbp_first,heart_rate_first,sbp_second,dbp_second,heart_rate_second,blood_staff_code,hours_since_last_ate_x10,random_glucose_x10,fasting_glucose_x10
csid,,,,,,,,,,,,,,,,,,,,,
2022-00229-1799093,0.122746,0.811057,0.592906,1.728535,-0.410683,-0.056127,1.780844,-1.604290,-0.884395,-0.449261,...,-0.190927,0.434371,0.418815,-0.340807,0.356325,0.112450,-0.473569,-0.654764,-0.272600,0.0
2022-00229-4744170,-1.250938,-0.168420,-0.422900,0.301515,-0.402491,-0.056127,-0.277026,-1.096393,-1.183793,-0.787761,...,-0.464590,-2.338602,0.093618,-0.295146,-1.779377,0.357120,-0.473569,-0.847423,0.093850,0.0
2022-00229-1082517,-0.727339,0.040177,0.259928,-0.802161,0.560934,-0.056127,0.472728,0.249563,0.161982,-0.686003,...,1.861547,1.560891,0.012318,1.987918,1.869113,0.112450,-1.218312,-0.462106,0.027223,0.0
2022-00229-3397875,1.103472,0.999186,-0.013410,0.773113,1.712047,-0.056127,0.331545,1.448354,1.390868,0.250701,...,-1.057527,-0.692149,-0.394179,-0.888742,-0.889501,-0.376892,1.228701,-0.751094,-0.372541,0.0
2022-00229-4372175,1.805687,1.875552,1.842670,1.767708,2.068840,-0.056127,2.404542,1.382637,1.962589,3.259559,...,-0.099706,0.174405,-0.963275,-0.386468,0.089362,-0.784676,0.803133,2.042459,-0.106032,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-00229-5565114,0.436560,-0.581341,-0.103120,0.316606,-0.240342,-0.056127,-0.783666,-0.087424,0.314348,0.772009,...,-0.692643,-0.518838,0.093618,-0.797419,-0.533551,0.030893,1.122309,-0.654764,0.027223,0.0
2022-00229-4902412,-1.082104,-0.489840,0.057002,-0.483336,-0.998423,-0.056127,0.273152,-1.188568,-1.268748,-0.039717,...,-1.011916,-0.172217,1.069210,-1.117048,0.000375,0.764905,-0.686353,-0.654764,-0.672364,0.0
2022-00229-4823882,2.495089,0.353427,1.444274,0.410068,1.829385,-0.056127,0.517444,0.972800,1.380486,0.211005,...,-0.510201,-0.865460,-1.776269,-0.797419,-0.266588,-1.437132,-1.005528,-0.654764,-0.006091,0.0


In [20]:
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_cat_imp = pd.DataFrame(imp_mode.fit_transform(X_cat), index = X_cat.index, columns = X_cat.columns)
X_cat_imp

,qc_warning,precipitation,region_is_urban,is_female,has_health_cover,has_own_home,has_private_toilet,has_phone,has_motor_vehicle,had_recent_holiday,tea_drank_in_past,alc_today,alc_drank_in_past,smoking_today,smoking_100_plus_cigs,has_diabetes,has_copd,diabetes_diag,diabetes_now,chd_diag,chd_now,stroke_or_tia_diag,hypertension_diag,stroke_or_tia_now,hypertension_now,taking_aspirin,taking_ace_i,taking_beta_blocker,taking_diuretics,taking_statins,taking_ca_antagonist,taking_chlor_metaformin,taking_insulin,rheum_heart_dis_diag,tb_diag,tb_now,emph_bronc_diag,emph_bronc_now,asthma_diag,asthma_now,cirrhosis_hep_diag,cirrhosis_hep_now,peptic_ulcer_diag,peptic_ulcer_now,gall_diag,gall_now,kidney_dis_diag,kidney_dis_now,fracture_diag,fracture_now,rheum_arthritis_diag,rheum_arthritis_now,psych_disorder_diag,neurasthenia_diag,head_injury_diag,neurasthenia_now,cancer_diag,cancer_now,mother_stroke,mother_heart_attack,mother_diabetes,mother_mental_disorder,father_stroke,mother_cancer,father_diabetes,father_heart_attack,father_mental_disorder,father_cancer,siblings_heart_attack,siblings_mental_disorder,children_diabetes,children_cancer,children_mental_disorder,tried_to_reduce_weight,used_blood_pressure_drugs,region_code_12,region_code_16,region_code_26,region_code_36,region_code_46,region_code_52,region_code_58,region_code_68,region_code_78,region_code_88,highest_education_0,highest_education_1,highest_education_2,highest_education_3,highest_education_4,highest_education_5,occupation_0,occupation_1,occupation_2,occupation_3,occupation_4,occupation_5,occupation_6,occupation_7,occupation_8,occupation_9,marital_status_0,marital_status_1,marital_status_2,marital_status_3,household_income_0,household_income_1,household_income_2,household_income_3,household_income_4,household_income_5,tea_year_0,tea_year_1,tea_year_2,tea_year_3,tea_year_4,tea_weekly_0.0,tea_weekly_1.0,tea_weekly_2.0,alcohol_category_1,alcohol_category_2,alcohol_category_3,alcohol_category_4,alcohol_category_5,alcohol_category_6,alc_year_0,alc_year_1,alc_year_2,alc_year_3,alc_year_4,alc_weekly_0.0,alc_weekly_1.0,alc_weekly_2.0,smoking_category_1,smoking_category_2,smoking_category_3,smoking_category_4,smoking_now_0,smoking_now_1,smoking_now_2,smoking_now_3,smoking_past_0.0,smoking_past_1.0,smoking_past_2.0,smoking_past_3.0,diet_freq_rice_0,diet_freq_rice_1,diet_freq_rice_2,diet_freq_rice_3,diet_freq_rice_4,diet_freq_wheat_0,diet_freq_wheat_1,diet_freq_wheat_2,diet_freq_wheat_3,diet_freq_wheat_4,diet_freq_other_staple_0,diet_freq_other_staple_1,diet_freq_other_staple_2,diet_freq_other_staple_3,diet_freq_other_staple_4,diet_freq_meat_0,diet_freq_meat_1,diet_freq_meat_2,diet_freq_meat_3,diet_freq_meat_4,diet_freq_poultry_0,diet_freq_poultry_1,diet_freq_poultry_2,diet_freq_poultry_3,diet_freq_poultry_4,diet_freq_fish_0,diet_freq_fish_1,diet_freq_fish_2,diet_freq_fish_3,diet_freq_fish_4,diet_freq_eggs_0,diet_freq_eggs_1,diet_freq_eggs_2,diet_freq_eggs_3,diet_freq_eggs_4,diet_freq_fresh_veg_0,diet_freq_fresh_veg_1,diet_freq_fresh_veg_2,diet_freq_fresh_veg_3,diet_freq_fresh_veg_4,diet_freq_soybean_0,diet_freq_soybean_1,diet_freq_soybean_2,diet_freq_soybean_3,diet_freq_soybean_4,diet_freq_preserved_veg_0,diet_freq_preserved_veg_1,diet_freq_preserved_veg_2,diet_freq_preserved_veg_3,diet_freq_preserved_veg_4,diet_freq_fresh_fruit_0,diet_freq_fresh_fruit_1,diet_freq_fresh_fruit_2,diet_freq_fresh_fruit_3,diet_freq_fresh_fruit_4,diet_freq_dairy_0,diet_freq_dairy_1,diet_freq_dairy_2,diet_freq_dairy_3,diet_freq_dairy_4,self_rated_health_0,self_rated_health_1,self_rated_health_2,self_rated_health_3,comparative_health_0,comparative_health_1,comparative_health_2,comparative_health_3,cancer_site_1.0,cancer_site_2.0,cancer_site_4.0,cancer_site_5.0,cancer_site_8.0,bowel_movement_freq_0,bowel_movement_freq_1,bowel_movement_freq_2,bowel_movement_freq_3,gum_bleed_freq_0,gum_bleed_freq_1,gum_bleed_freq_2,gum_bleed_freq_3,mother_still_alive_0,mother_still_alive_1,mother_still_alive_2,father_still_alive_0,fat

In [22]:
# Putting the data set together and setting NA v
X_tr = pd.concat([X_scaled_imp,X_cat_imp], axis = 1)
X_tr

,il8_x100000,tnfrsf9_x100000,tie2_x100000,mcp_3_x100000,cd40_l_x100000,il_1_alpha_x100000,cd244_x100000,egf_x100000,ang_1_x100000,il7_x100000,pgf_x100000,il6_x100000,adgrg1_x100000,mcp_1_x100000,crtam_x100000,cxcl11_x100000,mcp_4_x100000,trail_x100000,fgf2_x100000,cxcl9_x100000,cd8a_x100000,caix_x100000,ifn_beta_x100000,ada_x100000,cd4_x100000,nos3_x100000,il2_x100000,gal_9_x100000,vegfr_2_x100000,cd40_x100000,il18_x100000,gzmh_x100000,vegfc_x100000,lap_tgf_beta_1_x100000,cxcl1_x100000,tnfsf14_x100000,il33_x100000,tweak_x100000,pdgf_subunit_b_x100000,pdcd1_x100000,faslg_x100000,cd28_x100000,ccl19_x100000,mcp_2_x100000,ccl4_x100000,il_35_x100000,gal_1_x100000,pd_l1_x100000,cd27_x100000,cxcl5_x100000,il5_x100000,hgf_x100000,gzma_x100000,ho_1_x100000,cx3cl1_x100000,cxcl10_x100000,cd70_x100000,il10_x100000,tnfrsf12a_x100000,ccl23_x100000,cd5_x100000,ccl3_x100000,mmp7_x100000,arg1_x100000,ncr1_x100000,dcn_x100000,tnfrsf21_x100000,tnfrsf4_x100000,mic_a_b_x100000,ccl17_x100000,angpt2_x100000,ptn_x100000,cxcl12_x100000,ifn_gamma_x100000,lamp3_x100000,casp_8_x100000,icoslg_x100000,mmp12_x100000,cxcl13_x100000,pd_l2_x100000,vegfa_x100000,il4_x100000,il_21_x100000,il12rb1_x100000,il13_x100000,ccl20_x100000,tnf_x100000,klrd1_x100000,gzmb_x100000,cd83_x100000,il12_x100000,csf_1_x100000,age_at_study_date_x100,household_size,tea_years_ago,alc_years_ago,smoking_today_amount,smoking_last_hour_amount,diabetes_diag_age,chd_diag_age,stroke_or_tia_diag_age,hypertension_diag_age,tb_diag_age,emph_bronc_diag_age,asthma_diag_age,cirrhosis_hep_diag_age,peptic_ulcer_diag_age,gall_diag_age,kidney_dis_diag_age,fracture_diag_age,rheum_arthritis_diag_age,psych_disorder_diag_age,neurasthenia_diag_age,head_injury_diag_age,cancer_diag_age,blood_transfusions,paid_blood_donations,children,siblings,mother_current_age,mother_age_at_death,father_current_age,father_age_at_death,siblings_stroke,siblings_diabetes,siblings_cancer,met,met_hours,met_work,met_work_hours,age_25_weight_jin,standing_height_mm,sitting_height_mm,waist_mm,hip_mm,waist_hip_ratio,weight_kg_x10,bmi_calc,impedance_ohms,fat_percent_x10,weight_etc_staff_code,sbp_mean,dbp_mean,heart_rate_mean,sbp_first,dbp_first,heart_rate_first,sbp_second,dbp_second,heart_rate_second,blood_staff_code,hours_since_last_ate_x10,random_glucose_x10,fasting_glucose_x10,qc_warning,precipitation,region_is_urban,is_female,has_health_cover,has_own_home,has_private_toilet,has_phone,has_motor_vehicle,had_recent_holiday,tea_drank_in_past,alc_today,alc_drank_in_past,smoking_today,smoking_100_plus_cigs,has_diabetes,has_copd,diabetes_diag,diabetes_now,chd_diag,chd_now,stroke_or_tia_diag,hypertension_diag,stroke_or_tia_now,hypertension_now,taking_aspirin,taking_ace_i,taking_beta_blocker,taking_diuretics,taking_statins,taking_ca_antagonist,taking_chlor_metaformin,taking_insulin,rheum_heart_dis_diag,tb_diag,tb_now,emph_bronc_diag,emph_bronc_now,asthma_diag,asthma_now,cirrhosis_hep_diag,cirrhosis_hep_now,peptic_ulcer_diag,peptic_ulcer_now,gall_diag,gall_now,kidney_dis_diag,kidney_dis_now,fracture_diag,fracture_now,rheum_arthritis_diag,rheum_arthritis_now,psych_disorder_diag,neurasthenia_diag,head_injury_diag,neurasthenia_now,cancer_diag,cancer_now,mother_stroke,mother_heart_attack,mother_diabetes,mother_mental_disorder,father_stroke,mother_cancer,father_diabetes,father_heart_attack,father_mental_disorder,father_cancer,siblings_heart_attack,siblings_mental_disorder,children_diabetes,children_cancer,children_mental_disorder,tried_to_reduce_weight,used_blood_pressure_drugs,region_code_12,region_code_16,region_code_26,region_code_36,region_code_46,region_code_52,region_code_58,region_code_68,region_code_78,region_code_88,highest_education_0,highest_education_1,highest_education_2,highest_education_3,highest_education_4,highest_education_5,occupation_0,occupation_1,occupation_2,occupation_3,occupation_4,occupation_5,occupation_6,occupation_7,occupation_8,occupation_9,marital_status_0,marital_status_1,marital_status_2,marital_status_3,househo

In [23]:
X_tr.to_csv("D:/compiled_data/X_tr.csv")

y.to_csv("D:/compiled_data/y.csv")